In [13]:
import pandas as pd
import regex as re
from tqdm import tqdm

In [140]:
def extract_company_names(text):
    # Define the regex pattern for matching month names (full and abbreviated), and month numbers
    month_pattern = r'(?:January|February|March|April|May|June|July|August|September|October|November|December|' \
                    r'Jan|Feb|Mar|Apr|Jun|Jul|Aug|Sep|Oct|Nov|Dec|' \
                    r'0?[1-9]|1[0-2])'  # Full month names, abbreviations, or month numbers (01-12)
    
    # Define the pattern for day and year (optional day and required 4-digit year)
    day_pattern = r'(?:[ \n,/]*\d{1,2})?'  # Optional day (1-31), can be separated by space, newline, or slash
    year_pattern = r'[ \n,/]*\d{4}'  # 4-digit year
    
    # Final date pattern combining month, day (optional), and year
    date_pattern = f'{month_pattern}{year_pattern}'
    
    # Full pattern to match date ranges "{date} to {date}" or "{date} to Current"
    pattern = r'([^0-9\n]*\n*)?' \
              r'([^0-9\n]*\n*)?' \
              r'(' + date_pattern + r'[ \n]*to[ \n]*(?:' + date_pattern + r'|current))' \
              r'[ \n]*'\
              r'(.*\n*)?' \
              r'(.*\n)?'
    
    try :

      # Extract the matched patterns from the text
      matches = re.findall(pattern, text, re.IGNORECASE)

      # If no matches, return empty list
      if not matches:
          return []

      companies = []

      # print(matches)

      counter = -1
      
      for match in matches:
          # Collect surrounding lines and relevant info
          baseline = [line for line in match if line]
          if counter == -1:
            counter = len(baseline)
          baseliner = baseline[-counter:]
          # print(baseliner)
          final = ""
          for i in range(3):
            if (counter - i - 3 != 0):
              final += baseliner[i]
          companies.append(final)
    except Exception as e:
      companies = []
      print(f"Error in extracting companies for {text} : {e}, {matches}")
    return companies

In [149]:
def process_resumes():
    # Read the data from the CSV file
    data = pd.read_csv('extracted_resume_sections.csv')
    
    # Apply the extract_company_names function to the 'text' column
    result = {}

    # filter out based on condition of section
    data2 = data[data['Section'] == 'experience']

    # check if the data has empty values
    data2 = data2.dropna()

    data2.to_csv('data2.csv', index=False)

    # iterate over the rows and extract company names
    results = data2['Content'].apply(extract_company_names)

    # # for i, row in tqdm(data.iterrows()):
    # #     if row['Section'] == 'experience':
    # #         try:
    # #             result[row['Resume ID']] = extract_company_names(row['Content'])
    # #         except Exception as e:
    # #             print(f"An error occurred while processing Resume ID {row['Resume ID']}: {row['Content']}")
            
    
    # # make a new dataframe with column "Resume ID" and "Company"
    df = pd.DataFrame({'Resume ID': data2['Resume ID'], 'Company': list(results)})

    # # make a csv file from the new dataframe
    df.to_csv('companies.csv', index=False)

In [150]:
process_resumes()

Error in extracting companies for Finance Manager

February 2001

to
September 2014 : list index out of range, [('Finance Manager\n\n', '', 'February 2001\n\nto\nSeptember 2014', '', '')]
